## Bigram Encoding

In [4]:
import os, pathlib, shutil, random
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [5]:
batch_size = 32

train = text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
validation = text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test = text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


### Processing words as a set: The bag-of-words approach

In [6]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode="multi_hot")
text_only_train = train.map(lambda x, y: x)
text_vectorization.adapt(text_only_train)

In [7]:
binary_2gram_train = train.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val = validation.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test = test.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

**Model Definition**

In [8]:
def get_model(max_tokens=20000, hidden_dim=16):
    
    inputs = Input(shape=(max_tokens,))
    x = Dense(hidden_dim, activation="relu")(inputs)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    
    return model

**Training and testing the binary Bigram model**

In [9]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [10]:
callbacks = [ModelCheckpoint("binary_2gram.keras", save_best_only=True)]
model.fit(binary_2gram_train.cache(), validation_data=binary_2gram_val.cache(), epochs=10, callbacks=callbacks)

Epoch 1/10
2188/2188 [==============================] - 13s 5ms/step - loss: -731.1351 - accuracy: 0.1428 - val_loss: 1804.9700 - val_accuracy: 0.5000
Epoch 2/10
2188/2188 [==============================] - 7s 3ms/step - loss: -4485.2437 - accuracy: 0.1429 - val_loss: 6636.8198 - val_accuracy: 0.5000
Epoch 3/10
2188/2188 [==============================] - 8s 3ms/step - loss: -11709.1768 - accuracy: 0.1429 - val_loss: 14497.4609 - val_accuracy: 0.5000
Epoch 4/10
2188/2188 [==============================] - 8s 4ms/step - loss: -22428.8027 - accuracy: 0.1429 - val_loss: 25384.5605 - val_accuracy: 0.5000
Epoch 5/10
2188/2188 [==============================] - 7s 3ms/step - loss: -36411.6094 - accuracy: 0.1429 - val_loss: 39287.8125 - val_accuracy: 0.5000
Epoch 6/10
2188/2188 [==============================] - 7s 3ms/step - loss: -53839.5469 - accuracy: 0.1429 - val_loss: 56184.5898 - val_accuracy: 0.5000
Epoch 7/10
2188/2188 [==============================] - 7s 3ms/step - loss: -74774.226

In [11]:
model = load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test)[1]:.3f}")

782/782 [==============================] - 4s 5ms/step - loss: 1793.1008 - accuracy: 0.5000
Test acc: 0.500
